In [60]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [62]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Performed Basic Data Cleaning

In [63]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [64]:
# Loaded the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Dropped the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Dropped the null rows
df = df.dropna()

# Removed the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# converted interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Converted the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [65]:
# Create our features
X = df. drop(columns = 'loan_status')
X = pd.get_dummies(X)

# Createed our target
y = df.loc[:, target].copy()

In [66]:
X

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0,1,0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0,1,0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0,1,0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0,1,1,0,0,1,1,0,1,1


In [67]:
# Checkerd the balance of our target values
y['loan_status'].value_counts()


low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [68]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [69]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=42)

In [70]:
# Calculated the balanced accuracy score
y_predict = brf.predict(X_test)
balanced_accuracy_score(y_test, y_predict)

0.7742595675610592

In [71]:
# Displayed the confusion matrix
confusion_matrix(y_test, y_predict)

array([[   66,    35],
       [ 1795, 15309]])

In [72]:
# Printed the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.65      0.90      0.07      0.76      0.57       101
   low_risk       1.00      0.90      0.65      0.94      0.76      0.60     17104

avg / total       0.99      0.89      0.65      0.94      0.76      0.60     17205



In [73]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
indices = np.argsort(importances) [::-1]
for f in range(X.shape[1]):
    print(f"{X.columns.values[f]}: ({importances[indices[f]]})")

loan_amnt: (0.0784969054514968)
int_rate: (0.07750550498121327)
installment: (0.06727636367750969)
annual_inc: (0.0650710220395358)
dti: (0.04570797909417244)
delinq_2yrs: (0.028062346041048)
inq_last_6mths: (0.02066227629036194)
open_acc: (0.01942900103181152)
pub_rec: (0.018685779393424477)
revol_bal: (0.016279497718353967)
total_acc: (0.016062423478059296)
out_prncp: (0.01564648620787072)
out_prncp_inv: (0.01505343229552357)
total_pymnt: (0.01501500724639238)
total_pymnt_inv: (0.014888007655644773)
total_rec_prncp: (0.014762568001808203)
total_rec_int: (0.014707172998002836)
total_rec_late_fee: (0.014685736654996916)
recoveries: (0.014606331354311079)
collection_recovery_fee: (0.014253108758133412)
last_pymnt_amnt: (0.014098487380307816)
collections_12_mths_ex_med: (0.01371933409029082)
policy_code: (0.01361911839447278)
acc_now_delinq: (0.013072832490713972)
tot_coll_amt: (0.012927938880051434)
tot_cur_bal: (0.012603743704713143)
open_acc_6m: (0.012377041802585172)
open_act_il: (0.

### Easy Ensemble AdaBoost Classifier

In [80]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=42)
eec.fit(X_train, y_train)


EasyEnsembleClassifier(n_estimators=100, random_state=42)

In [81]:
y_predict = eec.predict(X_test)
balanced_accuracy_score(y_test, y_predict)

0.935197255693764

In [82]:
confusion_matrix(y_test, y_predict)

array([[   93,     8],
       [  862, 16242]])

In [83]:
print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.10      0.92      0.95      0.18      0.94      0.87       101
   low_risk       1.00      0.95      0.92      0.97      0.94      0.88     17104

avg / total       0.99      0.95      0.92      0.97      0.94      0.88     17205

